<a href="https://colab.research.google.com/github/arushirai1/grun_test/blob/master/ExtractiveSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jump to bottom to work with the BertSum model





In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/tennis_articles.csv', encoding = "ISO-8859-1")


In [ ]:
df[df['article_id'] == 1]['article_text'][0]

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net. So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [ ]:
sentences[:5]

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.",
 "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."]

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
word_embeddings = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word]=coefs

In [ ]:
len(word_embeddings)

400000

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [ ]:
!pip install transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
tokenizer = AutoTokenizer.from_pretrained("remi/bertabs-finetuned-cnndm-extractive-abstractive-summarization")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("https://s3.amazonaws.com/models.huggingface.co/bert/remi/bertabs-finetuned-cnndm-extractive-abstractive-summarization/config.json")


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Special tokens have been added in the vocabulary, make sure the associated word emebedding are fine-tuned or trained.


In [ ]:
model = AutoModelWithLMHead.from_pretrained("remi/bertabs-finetuned-cnndm-extractive-abstractive-summarization")


In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [ ]:
tokenizer(["Test test test", " Blah blah."])

{'input_ids': [[101, 3231, 3231, 3231, 102], [101, 27984, 27984, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

# BertSum
1. Mount Colab onto your drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. Add pretrained weights to your drive
https://drive.google.com/file/d/1kKWoV0QCbeIuFt85beQgJ4v0lujaXobJ/view

3. Unzip into a directory

In [ ]:
!ls drive/My\ Drive/*.zip
!mkdir drive/My\ Drive/bertsum_pretrained

'drive/My Drive/bertext_cnndm_transformer.zip'
'drive/My Drive/CS430LectureHandouts (1).zip'
mkdir: cannot create directory ‘drive/My Drive/bertsum_pretrained’: File exists


In [ ]:
!unzip drive/My\ Drive/bertext_cnndm_transformer.zip -d drive/My\ Drive/bertsum_pretrained


Archive:  drive/My Drive/bertext_cnndm_transformer.zip
replace drive/My Drive/bertsum_pretrained/bertext_cnndm_transformer.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# to confirm path exists
!ls drive/My\ Drive/bertsum_pretrained/bertext_cnndm_transformer.pt

'drive/My Drive/bertsum_pretrained/bertext_cnndm_transformer.pt'


In [ ]:
!git clone https://github.com/nlpyang/PreSumm

fatal: destination path 'PreSumm' already exists and is not an empty directory.


In [ ]:
!mkdir models
!cp -r PreSumm/src/models/* models/

mkdir: cannot create directory ‘models’: File exists


In [ ]:
!ls models

adam.py		encoder.py   model_builder.py  predictor.py	reporter.py
data_loader.py	__init__.py  neural.py	       __pycache__	trainer_ext.py
decoder.py	loss.py      optimizers.py     reporter_ext.py	trainer.py


In [ ]:
!pip install pytorch-transformers

In [ ]:
import torch
state_dict = torch.load('drive/My Drive/bertsum_pretrained/bertext_cnndm_transformer.pt')

In [ ]:
state_dict.keys()

dict_keys(['model', 'opt', 'optim'])

In [ ]:
optim = state_dict['optim']


In [ ]:
state_dict['opt']


Namespace(accum_count=2, alpha=0.6, batch_size=3000, beam_size=5, bert_data_path='../../bert_data_cnndm_final/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.1, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0, 1, 2], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../../logs/ext_bert_cnndm', lr=0.002, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=150, max_pos=512, max_tgt_len=140, min_length=15, mode='train', model_path='../../models/ext_bert_cnndm', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=50, report_rouge=True, result_path='../../results/cnndm', save_checkpoint_steps=1000, seed=666, sep_optim=False, share_emb=False, task='ext', temp_dir='../../temp', te

In [ ]:
from models.model_builder import ExtSummarizer

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

args = AttrDict()
args.update({'large': True, 'temp_dir':'./', 'finetune_bert': False, 'encoder':'baseline', 'ext_ff_size': 3072, 'ext_heads': 10, 'max_pos': 512, 'ext_hidden_size':768, 'ext_layers':8, 'ext_dropout':0.2})

In [ ]:
model=ExtSummarizer(args, 'cuda', state_dict)

AssertionError: ignored

In [ ]:
torch.cuda.empty_cache()

In [ ]:
pip install bert-extractive-summarizer

     |████████████████████████████████| 778kB 6.1MB/s 
     |████████████████████████████████| 3.0MB 40.3MB/s 
     |████████████████████████████████| 1.1MB 43.1MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
  Created wheel for bert-extractive-summarizer: filename=bert_extractive_summarizer-0.5.0-cp36-none-any.whl size=16418 sha256=2ef5d1e8f181473d65277fe51d6363edfcc0342ba175241f601bedf8c3c51959
  Stored in directory: /root/.cache/pip/wheels/4c/89/f7/13d71ca6b8e825191f2c5bfbe91f425cf12d2eeca07935cf13
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b3e8bf0956d3477fc39290651b464cdbfd264e4fd295b08b42e89ac3b9288330
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built bert-extractive-summarizer sacremoses


In [ ]:
from summarizer import Summarizer

body = 'Text body that you want to summarize with BERT'
body2 = 'Something else you want to summarize with BERT'
model = Summarizer()
model(body)
model(body2)

'Something else you want to summarize with BERT'

In [ ]:

body = df[df['article_id']==4]['article_text'][4-1]

In [ ]:
model(body)

'Kei Nishikori will try to end his long losing streak in ATP finals and Kevin Anderson will go for his second title of the year at the Erste Bank Open on Sunday. Nishikori leads Anderson 4-2 on career matchups, but the South African won their only previous meeting this year. Against Verdasco, Anderson hit nine of his 19 aces in the opening set.'

"Kei Nishikori will try to end his long losing streak in ATP finals and Kevin Anderson will go for his second title of the year at the Erste Bank Open on Sunday. The fifth-seeded Nishikori reached his third final of 2018 after beating Mikhail Kukushkin of Kazakhstan 6-4, 6-3 in the semifinals. A winner of 11 ATP events, Nishikori hasn't triumphed since winning in Memphis in February 2016. He has lost eight straight finals since. The second-seeded Anderson defeated Fernando Verdasco 6-3, 3-6, 6-4. Anderson has a shot at a fifth career title and second of the year after winning in New York in February. Nishikori leads Anderson 4-2 on career matchups, but the South African won their only previous meeting this year. With a victory on Sunday, Anderson will qualify for the ATP Finals. Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London next month. Nishikori held serve throughout against Kukushkin, who came through qualif